Lendo o arquivo com os dados

In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
nome_do_arquivo = 'Market variables/Bitcoin Historical Data (1).csv' 
dados = pd.read_csv(nome_do_arquivo)
dados

,Date,Price,Open,High,Low,Vol.,Change %
0,10/31/2023,"34,299.4","34,478.9","34,604.3","34,053.9",42.95K,-0.52%
1,10/30/2023,"34,477.9","34,526.7","34,853.3","34,089.5",45.20K,-0.14%
2,10/29/2023,"34,526.7","34,082.6","34,737.9","33,937.6",27.72K,1.30%
3,10/28/2023,"34,082.6","33,901.6","34,387.8","33,867.3",22.00K,0.53%
4,10/27/2023,"33,901.8","34,153.7","34,245.8","33,423.6",47.33K,-0.74%
...,...,...,...,...,...,...,...
4849,07/22/2010,0.1,0.1,0.1,0.1,2.16K,0.00%
4850,07/21/2010,0.1,0.1,0.1,0.1,0.58K,0.00%
4851,07/20/2010,0.1,0.1,0.1,0.1,0.26K,0.00%
4852,07/19/2010,0.1,0.1,0.1,0.1,0.57K,0.00%


Agora renomear as colunas necessárias e mudar e adicionar a variável alvo.

In [3]:
dados = dados.rename(columns={
    'Price': 'close',
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Vol.': 'vol'
})

Data set:

Os dados do dataset são refetentes aos dados diários do ativo, ou seja, temos o open, high, low, volume e Price_close do ativo depois do fechamento do dia. 
O código a seguir tem a intenção de, com os dados do dia anterior, prever alta ou baixa no dia seguinte. 

Para que nossos dados funcionem no problema em questão, deslocaremos o "Change" para baixo. Inicialmente esse change é uma comparação entre o dia atual e o anterior. Se houve subida, teremos um valor positivo, e caso contrario será negativo. Sendo assim, será necessario deslocar essa coluna pra baixo. Assim será nossa variavel alvo.

Depois de arrumar a variável alvo no código, deslocaremos todas as linhas para baixo com o intúito de fazer a ultima das modificações relevantes nos dados. Como nosso objetivo é prever se o dia de hj, que acabou de começar, vai ser uma alta ou baixa. Para tal, nossos dados atuais serão referentes ao dia de ontem. Por esse motivo que é necessário deslocar a tabela para baixo. Dessa forma os ultimos dados de hj serão referentes ao dia de ontem.

In [4]:
# Convertendo a coluna 'Change %' para float e verificando se é positivo ou negativo
dados['Change %'] = dados['Change %'].str.rstrip('%').astype('float')  # Removendo o símbolo de porcentagem e convertendo para float

# Atualizando os valores da coluna 'Change %' com base nas condições
dados['Change %'] = dados['Change %'].apply(lambda x: 1 if x >= 0 else -1)

#Shit na coluna alvo pra que ela fique coerente. O alvo indica oq o algorítimo deve prever.
dados['Change %'] = dados['Change %'].shift(1)

#Aqui damos shift em todo o data set. Isso é feito para que os dados fiquem coerentes. Tipo, no dia 2 da tabela, teremos todas as infos do dia 1.
date_column = dados['Date']
dados = dados.shift(1)
dados['Date'] = date_column
 
#Tirando os valores NaN
dados = dados.dropna()
 
# Renomear a coluna 'Change %' para 'target'
dados = dados.rename(columns={'Change %': 'predicted'})

# Definir a coluna "Date" como índice
dados.set_index('Date', inplace=True)

colunas_para_ajustar = ['close', 'open', 'high', 'low']  # Substitua pelos nomes reais das colunas no seu DataFrame

# Remover a vírgula e converter para o tipo numérico
for coluna in colunas_para_ajustar:
    dados[coluna] = dados[coluna].str.replace(',', '')#.astype(float)


#Converter valores de volume de str para float
def convert_to_numeric(value):
    if 'B' in value:
        return float(value.replace('B', '')) * 1000000000  # Convertendo para bilhões
    elif 'M' in value:
        return float(value.replace('M', '')) * 1000000    # Convertendo para milhões
    elif 'K' in value:
        return float(value.replace('K', '')) * 1000       # Convertendo para milhares
    else:
        return float(value)

# Aplicar a função para converter os valores na coluna 'vol' para números
dados['vol'] = dados['vol'].apply(convert_to_numeric)

dados = dados.iloc[::-1]

dados = dados.drop(dados.index[:99])
#dados.reset_index(drop=True, inplace=True)

dados

,close,open,high,low,vol,predicted
Date,,,,,,
10/25/2010,0.2,0.1,0.2,0.1,19220.0,1.0
10/26/2010,0.2,0.2,0.2,0.2,65610.0,1.0
10/27/2010,0.2,0.2,0.2,0.2,21530.0,1.0
10/28/2010,0.2,0.2,0.2,0.2,28690.0,1.0
10/29/2010,0.2,0.2,0.2,0.2,26710.0,1.0
...,...,...,...,...,...,...
10/25/2023,34153.7,34497.6,34826.0,33749.9,56620.0,-1.0
10/26/2023,33901.8,34153.7,34245.8,33423.6,47330.0,1.0
10/27/2023,34082.6,33901.6,34387.8,33867.3,22000.0,1.0


In [5]:
dados.to_csv('dados_w_market_features.csv', index=False)